# Nettoyage et préparation de données avec Skrub
Ce notebook vous guide à travers les principales fonctionnalités de Skrub : nettoyage, vectorisation, jointure floue et pipelines ML.

In [1]:
# Installation (à exécuter dans votre environnement si nécessaire)
# !pip install skrub pandas scikit-learn

## Chargement d'un jeu de données

In [2]:
import pandas as pd
from skrub.datasets import fetch_employee_salaries
from sklearn.model_selection import train_test_split

dataset = fetch_employee_salaries()
X, y = dataset.X, dataset.y

# séparation des données en apprentissage / test
x_train, x_test, y_train, y_test = train_test_split(X,y)


## Nettoyage automatique avec `Cleaner`

La classe Cleaner est un outil conçu pour simplifier le nettoyage des données tabulaires avant leur utilisation dans des modèles de machine learning. Elle automatise plusieurs étapes essentielles du prétraitement des données, ce qui permet de gagner du temps et de réduire les erreurs humaines.

Fonctionnalités principales de Cleaner :

- Détection et conversion des types de données : Cleaner identifie automatiquement les colonnes contenant des dates et les convertit en objets datetime, facilitant ainsi leur manipulation ultérieure.
- Gestion des valeurs manquantes : Il détecte les valeurs nulles ou manquantes et peut supprimer les colonnes contenant trop de valeurs nulles, améliorant ainsi la qualité des données.
- Suppression des colonnes peu informatives : Les colonnes avec une seule valeur unique ou une forte proportion de valeurs manquantes peuvent être automatiquement supprimées, réduisant ainsi la dimensionnalité inutile.

In [3]:
from skrub import Cleaner

cleaner = Cleaner()
x_train_clean = cleaner.fit_transform(x_train)


On peut afficher les transformations appliquées :

In [4]:
cleaner.all_processing_steps_

{'gender': [CleanNullStrings(), DropIfTooManyNulls(), ToStr()],
 'department': [CleanNullStrings(), DropIfTooManyNulls(), ToStr()],
 'department_name': [CleanNullStrings(), DropIfTooManyNulls(), ToStr()],
 'division': [CleanNullStrings(), DropIfTooManyNulls(), ToStr()],
 'assignment_category': [CleanNullStrings(), DropIfTooManyNulls(), ToStr()],
 'employee_position_title': [CleanNullStrings(),
  DropIfTooManyNulls(),
  ToStr()],
 'date_first_hired': [CleanNullStrings(), DropIfTooManyNulls(), ToDatetime()],
 'year_first_hired': [DropIfTooManyNulls()]}

In [5]:
# données initiales
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6921 entries, 2513 to 4476
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   gender                   6907 non-null   object
 1   department               6921 non-null   object
 2   department_name          6921 non-null   object
 3   division                 6921 non-null   object
 4   assignment_category      6921 non-null   object
 5   employee_position_title  6921 non-null   object
 6   date_first_hired         6921 non-null   object
 7   year_first_hired         6921 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 486.6+ KB


In [6]:
# données nettoyées
x_train_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6921 entries, 2513 to 4476
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   gender                   6907 non-null   object        
 1   department               6921 non-null   object        
 2   department_name          6921 non-null   object        
 3   division                 6921 non-null   object        
 4   assignment_category      6921 non-null   object        
 5   employee_position_title  6921 non-null   object        
 6   date_first_hired         6921 non-null   datetime64[ns]
 7   year_first_hired         6921 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 486.6+ KB


## TableReport

La classe TableReport est un outil interactif conçu pour faciliter l'analyse exploratoire des données tabulaires. Elle génère un rapport riche et structuré, offrant une vue d'ensemble complète d'un DataFrame, ce qui permet de mieux comprendre la structure et la qualité des données avant de les utiliser dans des modèles de machine learning.

In [7]:
from skrub import TableReport

report = TableReport(x_train_clean)


In [8]:
report

Processing column   8 / 8


,,,,,,,,


## Vectorisation avec `TableVectorizer`

Pour préparer les données en vue de l'entraînement d'un modèle de machine learning, il est nécessaire de convertir les variables catégorielles en représentations numériques. Skrub propose TableVectorizer pour automatiser ce processus 

In [9]:
from skrub import TableVectorizer

vec = TableVectorizer()
x_train_vec = vec.fit_transform(x_train_clean)


TableVectorizer détecte automatiquement les types de colonnes (numériques, catégorielles, dates) et applique les transformations appropriées, telles que l'encodage one-hot ou l'encodage de dates.

In [10]:
vec

TableVectorizer()

## Jointure floue avec `fuzzy_join`

Dans de nombreux cas, il est nécessaire de fusionner des tables contenant des clés similaires mais non identiques (par exemple, des noms de pays avec des variations orthographiques). Skrub propose la fonction fuzzy_join pour effectuer des jointures approximatives :

In [11]:
from skrub import fuzzy_join

left = pd.DataFrame({'Country': ['France', 'Italia', 'Georgia']})
right = pd.DataFrame({'Country Name': ['France', 'Italy', 'Germany'], 'Capital': ['Paris', 'Rome', 'Berlin']})

joined = fuzzy_join(left, right, left_on='Country', right_on='Country Name', add_match_info=True)
joined

,Country,Country Name,Capital,skrub_Joiner_distance,skrub_Joiner_rescaled_distance,skrub_Joiner_match_accepted
0,France,France,Paris,0.000000,0.000000,True
1,Italia,Italy,Rome,1.044348,0.751248,True
2,Georgia,Germany,Berlin,1.336161,0.961163,True


In [12]:
joined = fuzzy_join(left, right, left_on='Country', right_on='Country Name',max_dist=0.8, add_match_info=True)
joined

,Country,Country Name,Capital,skrub_Joiner_distance,skrub_Joiner_rescaled_distance,skrub_Joiner_match_accepted
0,France,France,Paris,0.000000,0.000000,True
1,Italia,Italy,Rome,1.044348,0.751248,True
2,Georgia,NaN,NaN,1.336161,0.961163,False


## Pipeline complet avec scikit-learn

Skrub s'intègre parfaitement avec scikit-learn, permettant la construction de pipelines complets pour le traitement des données et l'entraînement de modèles :

In [13]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor

pipeline = make_pipeline(
    Cleaner(),
    TableVectorizer(),
    RandomForestRegressor(random_state=42)
)

pipeline.fit(x_train, y_train)


Pipeline(steps=[('cleaner', Cleaner()), ('tablevectorizer', TableVectorizer()),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

In [14]:
from sklearn.metrics import root_mean_squared_error

predictions = pipeline.predict(x_test)
rmsea = root_mean_squared_error(y_test,predictions)
print(f"Le RMSEA sur les données de test est de : {rmsea}")

Le RMSEA sur les données de test est de : 9337.490279018954


Le processus de machine learning est encore plus simple qu'avec scikit-learn uniquement.

## Conclusion

Skrub est une bibliothèque puissante pour la préparation des données tabulaires en vue du machine learning. Elle simplifie les étapes de nettoyage, de transformation et de fusion des données, tout en s'intégrant harmonieusement avec les outils existants de l'écosystème Python. Que vous soyez data scientist, analyste ou ingénieur en machine learning, Skrub peut grandement faciliter votre travail sur des données réelles souvent désordonnées.

## Ressources supplémentaires

- Documentation officielle : https://skrub-data.org/
- Dépôt GitHub : https://github.com/skrub-data/skrub